In [1]:
import numpy as np
import pandas as pd
from deribit_api import RestClient
import matplotlib.pyplot as plt

In [2]:
from models import BSprice, BSgreeks, BSiv, otm, vvv, vvv_fitter, get_usd_curve

In [3]:
zrc = get_usd_curve()

In [4]:
client = RestClient()

In [5]:
alld = pd.DataFrame(client.getinstruments())

In [6]:
pullderibit=pd.DataFrame(client.getsummary('all'))

In [7]:
allderibit = alld.merge(pullderibit,on='instrumentName')

In [8]:
allderibit

,baseCurrency,contractSize,created_x,currency,expiration,instrumentName,isActive,kind,minTradeSize,optionType,...,low,markPrice,midPrice,openInterest,openInterestAmount,uIx,uPx,volume,volumeBtc,volumeUsd
0,BTC,NaN,2018-10-04 11:14:09 GMT,USD,2019-06-28 08:00:00 GMT,BTC-28JUN19-20000-C,True,option,0.1,call,...,0.0005,0.0002,,695.1,NaN,BTC-28JUN19,3289.39,15,15,NaN
1,BTC,NaN,2019-01-24 08:00:02 GMT,USD,2019-02-01 08:00:00 GMT,BTC-1FEB19-4250-P,True,option,0.1,put,...,0.243,0.2468,,9.0,NaN,index_price,3408.80,2,2,NaN
2,BTC,NaN,2019-01-04 16:46:13 GMT,USD,2019-09-27 08:00:00 GMT,BTC-27SEP19-1500-P,True,option,0.1,put,...,,0.0185,0.0185,903.5,NaN,BTC-28JUN19,3289.39,,,NaN
3,BTC,NaN,2018-07-27 22:09:09 GMT,USD,2019-03-29 08:00:00 GMT,BTC-29MAR19-10000-P,True,option,0.1,put,...,,1.9980,1.297,188.8,NaN,BTC-29MAR19,3335.58,,,NaN
4,BTC,NaN,2019-01-04 16:46:13 GMT,USD,2019-09-27 08:00:00 GMT,BTC-27SEP19-5500-C,True,option,0.1,call,...,0.045,0.0503,0.05025,26.7,NaN,BTC-28JUN19,3289.39,0.1,0.1,NaN
5,BTC,NaN,2018-09-28 09:49:43 GMT,USD,2019-06-28 08:00:00 GMT,BTC-28JUN19-12000-C,True,option,0.1,call,...,0.0005,0.0005,,216.1,NaN,BTC-28JUN19,3289.39,2,2,NaN
6,BTC,NaN,2018-11-27 13:19:33 GMT,USD,2019-06-28 08:00:00 GMT,BTC-28JUN19-1500-P,True,option,0.1,put,...,0.006,0.0080,0.008,747.7,NaN,BTC-28JUN19,3289.39,54.4,54.4,NaN
7,BTC,NaN,2019-01-18 08:07:57 GMT,USD,2019-02-22 08:00:00 GMT,BTC-22FEB19-5250-C,True,option,0.1,call,...,,0.0005,,45.9,NaN,index_price,3408.80,,,NaN
8,BTC,NaN,2018-11-27 13:16:50 GMT,USD,2019-03-29 08:00:00 GMT,BTC-29MAR19-3250-C,True,option,0.1,call,...,0.116,0.1148,0.11475,36.3,NaN,BTC-29MAR19,3335.58,0.6,0.6,NaN
9,BTC,NaN,2019-01-04 17:38:03 GMT,USD,2019-09-27 08:00:00 GMT,BTC-27SEP19-12000-C,True,option,0.1,call,...,0.0025,0.0028,0.00275,695.7,NaN,BTC-28JUN19,3289.39,6.5,6.5,NaN


In [9]:
columns = ['instrumentName','baseCurrency','optionType','expiration','strike','uPx','uIx','iR','bidPrice','askPrice','midPrice','openInterest','volume']
newcolumns = ['Name','Und','Ins','Expiry','Strike','uPx','uIx','iR','Bid','Ask','Mid','OI','Volume']
prices = allderibit[columns]
prices.columns = newcolumns
prices = prices.sort_values(['Expiry','Strike','Ins']).reset_index(drop=True)[:-1]
now = pd.datetime.now().strftime('%Y-%m-%d %H:%M')
prices['T'] = prices.apply(lambda x: round((pd.to_datetime(x['Expiry'])-pd.to_datetime('now')).total_seconds()/(365*24*60*60),4) ,axis=1)
prices['Expiry'] = prices["Expiry"].apply(lambda x: x.split(' ')[0])
prices['Volume'].replace('',0,inplace=True)
prices.replace('',np.nan,inplace=True)
prices['Ins'] = prices['Ins'].apply(lambda x : 'C' if x =='call' else ('P' if x =='put' else 'F'))
prices['OTM'] = prices.apply(lambda x :otm(x['uPx'],x['Strike'],x['Ins'],.001),axis=1)
prices['iR']=prices['T'].apply(lambda x:zrc(x))

sheets = prices.copy()

futures = sheets[sheets['Ins']=='F'].drop(['Strike','uIx','uPx','iR','OTM'],axis=1)
futures.reset_index(drop=True,inplace=True)
options = sheets[sheets['OTM']==True].dropna().drop('OTM',axis=1)
options.reset_index(inplace=True,drop=True)

options['BidVol'] = options.apply(lambda x : BSiv(x['uPx'], x['Strike'], x['T'], x['iR'], x['Bid']*x['uPx'], option = x['Ins']) ,axis=1)
options['MidVol'] = options.apply(lambda x : BSiv(x['uPx'], x['Strike'], x['T'], x['iR'], x['Mid']*x['uPx'], option = x['Ins']) ,axis=1)
options['AskVol'] = options.apply(lambda x : BSiv(x['uPx'], x['Strike'], x['T'], x['iR'], x['Ask']*x['uPx'], option = x['Ins']) ,axis=1)
options['VolSpread']= options['AskVol']-options['BidVol']
#options = options[options['AskVol']-options['BidVol']<.3]
options['Bid$'] = options['Bid']*options['uPx']
options['Mid$'] = options['Mid']*options['uPx']
options['Ask$'] = options['Ask']*options['uPx']
options['Vega'] = options.apply(lambda x : BSgreeks(x['uPx'], x['Strike'], x['T'], 0, x['MidVol'], option = 'C')[2]*.01,axis=1)
options['PV_factor']=np.exp(-options['iR']*options['T'])

In [10]:
optmats = []
T = []
Expiries = []
Ref = []
fitparams = []
volfitquality = []
pricefitquality = []

for i in options['T'].unique():
    mat = options[options['T']==i].copy()
    optmats.append(mat)
    t = mat.iloc[0]['T']
    Expiry = mat.iloc[0]['Expiry']
    Expiries.append(Expiry)
    T.append(t)
    ref = mat.iloc[0]['uPx']
    Ref.append(ref)
    try:
        fit=[round(i,4) for i in vvv_fitter(mat['Strike'][:],mat['MidVol'][:],mat['Vega'][:],t,ref)]
        fitparams.append(fit)
        mat['Fit'] = mat['Strike'].apply(lambda x : vvv(x,t,ref,*fit))
        volfitquality.append(round(np.abs(mat['Fit']-mat['MidVol']).mean(),4))
        mat['TV'] = mat.apply(lambda x: BSprice (ref, x['Strike'], t, x['iR'], x['Fit'], option = x['Ins']),axis=1)
        pricefitquality.append(round(np.abs(mat['TV']-mat['Mid$']).mean(),4))
    except:
        fit=['nofit']
        fitparams.append(fit)
        volfitquality.append(None)
        pricefitquality.append(None)
        continue

In [11]:
optmats[3]

,Name,Und,Ins,Expiry,Strike,uPx,uIx,iR,Bid,Ask,...,MidVol,AskVol,VolSpread,Bid$,Mid$,Ask$,Vega,PV_factor,Fit,TV
30,BTC-28JUN19-1500-P,BTC,P,2019-06-28,1500.0,3289.39,BTC-28JUN19,0.024801,0.0070,0.0090,...,0.776794,0.797047,0.041876,23.025730,26.315120,29.604510,1.590843,0.989851,0.782989,27.300018
31,BTC-28JUN19-2000-P,BTC,P,2019-06-28,2000.0,3289.39,BTC-28JUN19,0.024801,0.0245,0.0275,...,0.728097,0.741732,0.027551,80.590055,85.524140,90.458225,3.619750,0.989851,0.727955,85.473108
32,BTC-28JUN19-2500-P,BTC,P,2019-06-28,2500.0,3289.39,BTC-28JUN19,0.024801,0.0590,0.0625,...,0.685743,0.695585,0.019732,194.074010,199.830442,205.586875,5.894875,0.989851,0.685989,199.973764
33,BTC-28JUN19-3000-P,BTC,P,2019-06-28,3000.0,3289.39,BTC-28JUN19,0.024801,0.1160,0.1195,...,0.655672,0.663248,0.015154,381.569240,387.325672,393.082105,7.675277,0.989851,0.655479,387.179192
34,BTC-28JUN19-3500-C,BTC,C,2019-06-28,3500.0,3289.39,BTC-28JUN19,0.024801,0.1335,0.1375,...,0.636896,0.644805,0.015816,439.133565,445.712345,452.291125,8.404484,0.989851,0.635460,444.517771
35,BTC-28JUN19-4000-C,BTC,C,2019-06-28,4000.0,3289.39,BTC-28JUN19,0.024801,0.0850,0.0890,...,0.623827,0.632050,0.016468,279.598150,286.176930,292.755710,8.072118,0.989851,0.624221,286.491402
36,BTC-28JUN19-4500-C,BTC,C,2019-06-28,4500.0,3289.39,BTC-28JUN19,0.024801,0.0535,0.0575,...,0.618335,0.627703,0.018821,175.982365,182.561145,189.139925,7.063353,0.989851,0.619332,183.258379
37,BTC-28JUN19-5000-C,BTC,C,2019-06-28,5000.0,3289.39,BTC-28JUN19,0.024801,0.0340,0.0370,...,0.616913,0.625428,0.017157,111.839260,116.773345,121.707430,5.811640,0.989851,0.618447,117.656605
38,BTC-28JUN19-6000-C,BTC,C,2019-06-28,6000.0,3289.39,BTC-28JUN19,0.024801,0.0145,0.0155,...,0.620424,0.625052,0.009335,47.696155,49.340850,50.985545,3.560580,0.989851,0.622352,50.022522
39,BTC-28JUN19-7000-C,BTC,C,2019-06-28,7000.0,3289.39,BTC-28JUN19,0.024801,0.0070,0.0075,...,0.635842,0.639642,0.007679,23.025730,23.848077,24.670425,2.164135,0.989851,0.628530,22.312330


In [12]:
prices

,Name,Und,Ins,Expiry,Strike,uPx,uIx,iR,Bid,Ask,Mid,OI,Volume,T,OTM
0,BTC-1FEB19-2750-C,BTC,C,2019-02-01,2750.0,3408.80,index_price,0.023721,0.1890,0.1945,0.19175,0.0,0.0,0.0086,False
1,BTC-1FEB19-2750-P,BTC,P,2019-02-01,2750.0,3408.80,index_price,0.023721,NaN,0.0010,NaN,246.7,38.2,0.0086,True
2,BTC-1FEB19-3000-C,BTC,C,2019-02-01,3000.0,3408.80,index_price,0.023721,0.1175,0.1220,0.11975,1.0,0.4,0.0086,False
3,BTC-1FEB19-3000-P,BTC,P,2019-02-01,3000.0,3408.80,index_price,0.023721,0.0010,0.0025,0.00175,109.0,19.7,0.0086,True
4,BTC-1FEB19-3250-C,BTC,C,2019-02-01,3250.0,3408.80,index_price,0.023721,0.0515,0.0560,0.05375,51.4,9.7,0.0086,False
5,BTC-1FEB19-3250-P,BTC,P,2019-02-01,3250.0,3408.80,index_price,0.023721,0.0075,0.0085,0.00800,282.0,165.5,0.0086,True
6,BTC-1FEB19-3500-C,BTC,C,2019-02-01,3500.0,3408.80,index_price,0.023721,0.0105,0.0125,0.01150,123.5,59.7,0.0086,True
7,BTC-1FEB19-3500-P,BTC,P,2019-02-01,3500.0,3408.80,index_price,0.023721,0.0385,0.0415,0.04000,245.8,42.1,0.0086,False
8,BTC-1FEB19-3750-C,BTC,C,2019-02-01,3750.0,3408.80,index_price,0.023721,0.0005,0.0015,0.00100,270.8,3.9,0.0086,True
9,BTC-1FEB19-3750-P,BTC,P,2019-02-01,3750.0,3408.80,index_price,0.023721,0.1010,0.1060,0.10350,6.4,3.7,0.0086,False


In [ ]:
optmats